In [18]:
import pandas as pd 
import requests 
import os 
from dotenv import load_dotenv
from  datetime import datetime,timedelta
import pyarrow
import time
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

load_dotenv()
# Load API key from .env
API_KEY  = os.getenv("API_KEY")
LANGUAGE = "en-US"
BASE_URL = "https://api.themoviedb.org/3"
params = {"api_key": API_KEY}

# Load snowflake credentials from .env

conn = snowflake.connector.connect(
    user= os.getenv("user"),
    password=os.getenv("password"),
    account=os.getenv("account"),
    warehouse=os.getenv("warehouse"),
    database=os.getenv("database"),
    schema=os.getenv("schema")
)


In [22]:
pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

'2025-06-18 08:56'

In [8]:
def get_trending_show_ids():
    '''Extracts trending show ids from TMDB'''
    url =  f"{BASE_URL}/trending/tv/week?language=en-US"
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()["results"]
    trending_show_ids = [show['id'] for show in data]
    return trending_show_ids



In [9]:
def get_trending_show_metadata(trending_show_id_list):
    '''Gets metadata of trending shows of the week and adds them to a list'''
    show_metadata = []
    for show in trending_show_id_list:
        try:
            url = f"{BASE_URL}/tv/{show}?"  
            response = requests.get(url, params=params)
            data = response.json()
            show_metadata.append(data)
        except requests.RequestException as error:
            print(f"Error fetching show {show}: {error}")
            continue
        time.sleep(0.2) 

    return show_metadata  

In [10]:
def get_week_start_date():
    """Returns the Monday of the current week."""
    today = datetime.today().date()
    start_of_week = today - timedelta(days=today.weekday())  # Monday
    return start_of_week


In [11]:
def extract_tv_show_to_df(show_metadata_list):
    ''' loads tv show metadata to df where each row represents a single show'''
    load_date = datetime.today().date()
    start_of_week = get_week_start_date()
    dataframe_rows =[]
    for show in show_metadata_list:
            dataframe_rows.append({
            "load_date": load_date,
            "start_of_week": start_of_week,
            "tv_show_id": show['id'],
            "first_air_date": show["first_air_date"],
            "tv_show_status": show['status'],
            "tv_show_type": show['type'],
            "vote_average": show['vote_average'],
            "original_language": show['original_language'],
            "localized_name": show['name'],
            "original_name": show['original_name'],
            "number_of_episodes": show["number_of_episodes"],
            "number_of_seasons": show["number_of_seasons"],
            "vote_count": show['vote_count']})
            
    df = pd.DataFrame(dataframe_rows)
    df["merge_key"] = df["tv_show_id"].astype(str) + "_" + df["start_of_week_date"].astype(str)
    df.columns = [col.upper() for col in df.columns]
    return(df)



In [12]:
#one show can have many genres and one genre will have many shows
'''loads all generes of a show to a dataframe where each row represnets a genre'''
def extract_genres_to_df(show_metadata_list):
    load_date = datetime.today().date()
    start_of_week = get_week_start_date()
    dataframe_rows = []
    for show in show_metadata_list:
        for genre in show.get("genres", []):
            dataframe_rows.append({
                "load_date": load_date,
                "week_start_date": start_of_week,
                "tv_show_id": show["id"],
                "genre_id": genre["id"],
                "genre_name": genre["name"]
            })
    df = pd.DataFrame(dataframe_rows)
    df["merge_key"] = df["tv_show_id"].astype(str) + "_" + df["start_of_week_date"].astype(str)
    df.columns = [col.upper() for col in df.columns]

    return(df)




In [13]:
def extract_creators_to_df(show_metadata_list):
    '''loads all  creaters of a show to a dataframe where each row represents a creator'''
    dataframe_rows = []
    load_date =datetime.today().date()
    start_of_week = get_week_start_date()
    for show in show_metadata_list:
        for creator in show.get("created_by", []):
            dataframe_rows.append({
                "load_date":load_date,
                "week_start_date":start_of_week,
                "tv_show_id": show["id"],
                "creator_id": creator["id"],
                "creator_name": creator["name"]
            })
    df = pd.DataFrame(dataframe_rows)
    df["merge_key"] = df["tv_show_id"].astype(str) + "_" + df["start_of_week_date"].astype(str)
    df.columns = [col.upper() for col in df.columns]
    return(df)


In [14]:
def extract_networks_to_df(show_metadata_list):
    dataframe_rows = []
    load_date = datetime.today().date()
    start_of_week = get_week_start_date()
    for show in show_metadata_list:
        for network in show.get("networks", []):
            dataframe_rows.append({
                "load_date":load_date,
                "week_start_date":start_of_week,
                "tv_show_id": show["id"],
                "network_id": network["id"],
                "network_name": network["name"]
            })
    df = pd.DataFrame(dataframe_rows)
    df["merge_key"] = df["tv_show_id"].astype(str) + "_" + df["start_of_week_date"].astype(str)
    df.columns = [col.upper() for col in df.columns]

    return(df)


In [15]:
def extracts_spoken_languages_to_df(show_metadata_list):
    dataframe_rows = []
    load_date = datetime.today().date()
    start_of_week = get_week_start_date()
    for show in show_metadata_list:
        for languages in show.get("spoken_languages", []):
            dataframe_rows.append({
                "load_date": load_date,
                "week_start_date":start_of_week,
                "tv_show_id": show["id"],
                "language_name": languages["english_name"]})
    
    df["merge_key"] = df["tv_show_id"].astype(str) + "_" + df["start_of_week_date"].astype(str)
    df = pd.DataFrame(dataframe_rows)
    df.columns = [col.upper() for col in df.columns]
    return(df)



In [17]:
def main():
    trending_show_id_list = get_trending_show_ids()
    get_week_start_date()
    show_metadata_list = get_trending_show_metadata(trending_show_id_list)
    tv_show_df = extract_tv_show_to_df(show_metadata_list)
    genre_df = extract_genres_to_df(show_metadata_list)
    creator_df = extract_creators_to_df(show_metadata_list)
    networks_df = extract_networks_to_df(show_metadata_list)
    spoken_language_df = extracts_spoken_languages_to_df(show_metadata_list)



    #Upload to Snowflake

    write_pandas(conn, tv_show_df, "TV_SHOW")
    write_pandas(conn, genre_df, "GENRES")
    write_pandas(conn, creator_df, "CREATORS")
    write_pandas(conn, networks_df, "NETWORKS")
    write_pandas(conn, spoken_language_df, "SPOKEN_LANGUAGE")
    

if __name__ == "__main__":
    main()

     LOAD_DATE START_OF_WEEK  TV_SHOW_ID FIRST_AIR_DATE    TV_SHOW_STATUS  \
0   2025-06-17    2025-06-16      221300     2023-05-25  Returning Series   
1   2025-06-17    2025-06-16      247718     2025-03-30  Returning Series   
2   2025-06-17    2025-06-16      203367     2025-06-07  Returning Series   
3   2025-06-17    2025-06-16       60625     2013-12-02  Returning Series   
4   2025-06-17    2025-06-16      100088     2023-01-15  Returning Series   
5   2025-06-17    2025-06-16       37854     1999-10-20  Returning Series   
6   2025-06-17    2025-06-16      241554     2025-05-15  Returning Series   
7   2025-06-17    2025-06-16       57243     2005-03-26             Ended   
8   2025-06-17    2025-06-16        1396     2008-01-20             Ended   
9   2025-06-17    2025-06-16      232766     2025-06-06             Ended   
10  2025-06-17    2025-06-16        1399     2011-04-17             Ended   
11  2025-06-17    2025-06-16       83867     2022-09-21             Ended   